Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [ ]:
#Update your token
STUDENT_TOKEN = 'GROUP 1'

# Base Code

There should be no need to change the code below.

In [ ]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_board(self):
    return json.loads(self.state)

  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('Server says there is a draw')
      else:
        print('Server says your agent ' + ('WON' if game.player == player else 'LOST'))
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      print(move)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

In [ ]:
def game_loop_test(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    id = new_game(game_type, multi_player)

  player = join_game(game_type, id)


  game = game_state(game_type, id, GameClass)
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    if game.is_end():
      if game.player == '-':
        return 0
      else:
        if game.player == player:
          return 1
        else: 
          return 0
      return
    if game.player == player:
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [ ]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class Clobber(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def print_game(self):
    for row in self.get_board():
      print(row)


# Random Solver

Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver".

Solvers need to return a list or tuple of 3 elements: row, col and dir. Dir can be 'R', 'L', 'U', 'D'

In [ ]:
def random_solver(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  opts = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          opts.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          opts.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          opts.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          opts.append((row, col, 'L'))
  return opts[random.randint(0, len(opts) - 1)]

This code is the one that starts the game. In different games you will need to change the class (Clobber is the one for clobber game).

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [ ]:
game_loop(random_solver, Clobber, 'clobber', multi_player=False, id=None)

# Solver V12 (The Chosen One)

In [ ]:
def number_of_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def available_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  a_moves = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          a_moves.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          a_moves.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          a_moves.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          a_moves.append((row, col, 'L'))
  return a_moves

def move_preview_v12(input_board, player, move):
  board = deepcopy(input_board)
  board[move[0]][move[1]] = '.'
  if move[2] == 'D':
    board[move[0] + 1][move[1]] = player
  if move[2] == 'R':
    board[move[0]][move[1] + 1] = player
  if move[2] == 'U':
    board[move[0] - 1][move[1]] = player
  if move[2] == 'L':
    board[move[0]][move[1] - 1] = player
  return board

def solver_v12(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  value, move, adv = max_value_v12(board, 3, player, other_player)
  return move

def max_value_v12(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return 1, None, other_moves
    else:
      return -1, None, other_moves
  if my_moves == 0:
    return -1, None, other_moves
  if other_moves == 0:
    return 1, None, 0
  win_options = {}
  lose_with_honor = {}
  for action in available_moves(board, player, other_player):
    preview = move_preview_v12(board, player, action)
    v2, a2, m2 = min_value_v12(preview, depth-1, other_player, player)
    if v2 == 1:
      win_options[action] = number_of_moves(preview, other_player, player)
    elif v2 == -1:
      lose_with_honor[action] = number_of_moves(preview, other_player, player)
  if len(win_options) == 0:
    move = min(lose_with_honor, key=lose_with_honor.get)
    advantage_moves = lose_with_honor[move]
  else:
    move = min(win_options, key=win_options.get)
    advantage_moves = win_options[move]
  value = v2
  return value, move, advantage_moves
  
def min_value_v12(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return -1, None, my_moves
    else:
      return 1, None, my_moves
  if my_moves == 0:
    return 1, None, 0
  if other_moves == 0:
    return -1, None, my_moves
  win_options = {}
  lose_with_honor = {}
  for action in available_moves(board, player, other_player):
    preview = move_preview_v12(board, player, action)
    v2, a2, m2 = max_value_v12(preview, depth-1, other_player, player)
    if v2 == -1:
      win_options[action] = number_of_moves(preview, other_player, player)
    elif v2 == 1:
      lose_with_honor[action] = number_of_moves(preview, other_player, player)
  if len(win_options) == 0:
    move = min(lose_with_honor, key=lose_with_honor.get)
    advantage_moves = lose_with_honor[move]
  else:
    move = min(win_options, key=win_options.get)
    advantage_moves = win_options[move]
  value = v2
  return value, move, advantage_moves

# Tester

In [ ]:
game_loop(solver_v12, Clobber, 'clobber', multi_player=False, id=None)

In [ ]:
i = 0
won = 0
while i < 10:
  won += game_loop_test(solver_v12, Clobber, 'clobber', multi_player=False, id=None)
  i += 1
  print(i)
print("The algorithm won ", won, " times!")